##1. 설치

In [1]:
!pip install fastapi uvicorn nest-asyncio flask pyngrok flask_ngrok

In [2]:
!ngrok config add-authtoken 30s47Zrpt0z8JIXJRCLMMklctLS_43GWGBsYeAL5k558po9Xo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


https://dashboard.ngrok.com/agents

## 2. flask

In [1]:
template = """
<!DOCTYPE html>
<html>
<head>
    <title>주가 예측</title>
</head>
<body>
    <h2>종가 예측 (t-1, t-2, t-3 → t)</h2>

    <h3>1. 모델 학습</h3>
    <form action="/train" method="post">
        티커: <input type="text" name="ticker" value="AAPL"><br>
        기간: <input type="text" name="period" value="6mo"><br>
        <button type="submit">학습하기</button>
    </form>

    <h3>2. 예측하기</h3>
    <form action="/predict" method="post">
        t-1: <input type="text" name="t1"><br>
        t-2: <input type="text" name="t2"><br>
        t-3: <input type="text" name="t3"><br>
        <button type="submit">예측</button>
    </form>
</body>
</html>
"""

In [3]:
import nest_asyncio
import yfinance as yf
import pickle
import threading
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from sklearn.ensemble import RandomForestRegressor

# 중복 실행 방지
nest_asyncio.apply()

# Flask 앱 정의
app = Flask(__name__)

@app.route('/')
def home():
    return render_template_string(template)

# ngrok으로 포트 5000 노출
public_url = ngrok.connect(5000)
print(f"🔗 Public URL: {public_url}")

def run_flask():
    app.run(port=5000)

# 쓰레드로 실행
threading.Thread(target=run_flask).start()


🔗 Public URL: NgrokTunnel: "https://3a0ba9f3561f.ngrok.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


##3.routing 추가

In [5]:
import nest_asyncio
import yfinance as yf
import pickle
import threading
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from sklearn.ensemble import RandomForestRegressor

# 중복 실행 방지
nest_asyncio.apply()

# Flask 앱 정의
app = Flask(__name__)

@app.route('/')
def home():
    return render_template_string(template)

@app.route('/train', methods=['POST'])
def train():
    ticker = request.form.get('ticker', 'AAPL')
    period = request.form.get('period', '6mo')

    df = yf.download(ticker, period=period)
    df = df[["Close"]]
    df["t-1"] = df["Close"].shift(1)
    df["t-2"] = df["Close"].shift(2)
    df["t-3"] = df["Close"].shift(3)
    df["target"] = df["Close"]
    df.dropna(inplace=True)

    X = df[["t-1", "t-2", "t-3"]]
    y = df["target"]

    model = RandomForestRegressor()
    model.fit(X, y)
    with open("model.pkl", "wb") as f:
        pickle.dump(model, f)

    return f"{ticker} 종목에 대해 {period} 기간으로 {len(X)}건 학습 완료"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        t1 = float(request.form.get('t1', 0))
        t2 = float(request.form.get('t2', 0))
        t3 = float(request.form.get('t3', 0))
        with open("model.pkl", "rb") as f:
            model = pickle.load(f)
        prediction = model.predict([[t1,t2,t3]])
        return f"예측된 종가: {prediction}"
    except ValueError as e:
        return f"입력 오류: {e}", 400

# ngrok으로 포트 5000 노출
public_url = ngrok.connect(5000)
print(f"🔗 Public URL: {public_url}")

def run_flask():
    app.run(port=5000)

# 쓰레드로 실행
threading.Thread(target=run_flask).start()
